# Deep Learning v1 Image Classification Development

## Objectives

- Design and implement a custom neural network from scratch
- Learn fundamental deep learning concepts through hands-on implementation
- Compare performance with shallow learning approaches
- Establish baseline for deep learning model improvements

## Setup and Imports

In [ ]:
!pip install torchinfo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
from torchinfo import summary
import os
import sys
from PIL import Image
import pickle
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix
import time
from typing import Dict, Any, List, Tuple

# Add parent directory to path for imports
sys.path.append('../..')
sys.path.append('..')

# Import from extracted modules
from src.classifier import DeepLearningV1Classifier
from src.trainer import DeepLearningV1Trainer
from src.model import DeepLearningV1
from src.config import DeepLearningV1Config
from src.data_loader import create_data_loaders, UnifiedDataset, get_transforms

# Import from ml_models_core
from ml_models_core.src.base_classifier import BaseImageClassifier
from ml_models_core.src.model_registry import ModelRegistry, ModelMetadata
from ml_models_core.src.utils import ModelUtils
from ml_models_core.src.data_loaders import get_unified_classification_data

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Plot settings
plt.style.use('default')
sns.set_palette('husl')

print("Setup complete - using extracted modules")
print("All required modules imported successfully")

In [ ]:
# Create configuration for deep learning v1
config = DeepLearningV1Config(
    image_size=(128, 128),  # Optimal 128x128 input size
    batch_size=8,
    learning_rate=0.001,
    num_epochs=30,
    patience=5,
    dropout_rate=0.5,
    random_seed=42
)

# Use the correct dataset path for this project  
dataset_path = "../../data/downloads/combined_unified_classification"

# Check if dataset exists
if not os.path.exists(dataset_path):
    print(f"Dataset not found at {dataset_path}")
    dataset_path = "data/downloads/combined_unified_classification"
    if not os.path.exists(dataset_path):
        print(f"Dataset not found at {dataset_path}")
        # Try to use the data manager to get/create the dataset
        try:
            from ml_models_core.src.data_manager import get_dataset_manager
            manager = get_dataset_manager()
            
            dataset_path = manager.get_dataset_path('combined_unified_classification')
            if not dataset_path:
                print("Creating unified classification dataset...")
                available_datasets = ['oxford_pets', 'kaggle_vegetables', 'street_foods', 'musical_instruments']
                dataset_path = manager.create_combined_dataset(
                    dataset_names=available_datasets,
                    output_name="combined_unified_classification",
                    class_mapping=None
                )
        except Exception as e:
            print(f"Error accessing unified dataset: {e}")
            # Fallback to oxford pets
            dataset_path = "data/downloads/oxford_pets"

print(f"Using dataset path: {dataset_path}")

# Create data loaders using extracted modules
try:
    print("Creating data loaders using extracted modules...")
    train_loader, val_loader, test_loader, class_names = create_data_loaders(dataset_path, config)
    
    print(f"✅ Data loaders created successfully")
    print(f"Found {len(class_names)} classes: {class_names[:5]}{'...' if len(class_names) > 5 else ''}")
    print(f"Training samples: {len(train_loader.dataset)}")
    print(f"Validation samples: {len(val_loader.dataset)}")
    print(f"Test samples: {len(test_loader.dataset)}")
    
    # Test loading a single batch to verify everything works
    print(f"\nTesting data loading with {config.image_size[0]}x{config.image_size[1]} images...")
    sample_batch = next(iter(train_loader))
    print(f"✅ Successfully loaded batch: {sample_batch[0].shape}, {sample_batch[1].shape}")
    print(f"✅ Memory-efficient loading working correctly")
    
except Exception as e:
    print(f"❌ Error creating data loaders: {e}")
    print("Check that the dataset path is correct and data exists")

# Memory check
import psutil
process = psutil.Process(os.getpid())
memory_mb = process.memory_info().rss / 1024 / 1024
print(f"Current memory usage: {memory_mb:.1f} MB")

In [ ]:
# Data loading and transforms are now handled by extracted modules
print("Data loading configuration:")
print(f"  Image size: {config.image_size}")
print(f"  Batch size: {config.batch_size}")
print(f"  Data augmentation: Random flips, rotations, color jittering")
print(f"  Normalization: ImageNet pretrained values")
print(f"  Memory-efficient: On-demand image loading from paths")

# Display data loader information
print(f"\nDataLoader details:")
print(f"  Training batches: {len(train_loader)}")
print(f"  Validation batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

# Verify transforms are working by checking a sample
sample_images, sample_labels = next(iter(train_loader))
print(f"\nSample batch verification:")
print(f"  Image tensor shape: {sample_images.shape}")
print(f"  Label tensor shape: {sample_labels.shape}")
print(f"  Image value range: [{sample_images.min():.3f}, {sample_images.max():.3f}]")
print(f"  Labels: {sample_labels[:5].tolist()}")

print(f"\n✅ Data pipeline ready for training with extracted modules")

# Clean up sample to free memory
del sample_images, sample_labels
import gc
gc.collect()

In [ ]:
# Visualize sample images
def visualize_batch(data_loader, class_names, title="Sample Images"):
    """Visualize a batch of images."""
    data_iter = iter(data_loader)
    images, labels = next(data_iter)
    
    # Denormalize images for visualization
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    images_denorm = images * std + mean
    images_denorm = torch.clamp(images_denorm, 0, 1)
    
    fig, axes = plt.subplots(2, 4, figsize=(12, 6))
    axes = axes.ravel()
    
    for i in range(min(8, len(images))):
        img = images_denorm[i].permute(1, 2, 0)
        axes[i].imshow(img)
        axes[i].set_title(f'{class_names[labels[i]]}')
        axes[i].axis('off')
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

visualize_batch(train_loader, class_names, "Training Samples from Unified Dataset")

## Neural Network Architecture Design

Now let's design our custom CNN architecture. We'll create a modular design with configurable depth and features.

In [ ]:
# Model architecture is now in the extracted modules
print("Using extracted DeepLearningV1 model from src.model module")
print("Model features:")
print("- Custom CNN architecture for image classification")
print("- Optimized for 128x128 input images")
print("- 5 convolutional blocks with batch normalization")
print("- Progressive channel increase: 3 → 32 → 64 → 128 → 256 → 512")
print("- Global average pooling for dimensionality reduction")
print("- Dropout regularization")
print("- Configurable number of classes")
print("- Kaiming weight initialization")

# Display model configuration
print(f"\nModel configuration:")
print(f"  Input channels: {config.input_channels}")
print(f"  Dropout rate: {config.dropout_rate}")
print(f"  Number of classes: {len(class_names)}")
print(f"  Input size: {config.image_size}")

print(f"\n✅ Model architecture ready from extracted modules")

In [ ]:
# Create model instance using extracted modules
print("Creating model using extracted DeepLearningV1 class...")

# Create model with dynamic class discovery
num_classes = len(class_names)
pytorch_model = DeepLearningV1(
    num_classes=num_classes,
    input_channels=config.input_channels,
    dropout_rate=config.dropout_rate
).to(device)

print(f"Model created for {num_classes} classes with {config.image_size[0]}x{config.image_size[1]} input")
print(f"Classes: {class_names[:5]}{'...' if len(class_names) > 5 else ''}")

# Print model summary
print("\nModel Architecture Summary:")
try:
    print(summary(pytorch_model, input_size=(config.batch_size, 3, config.image_size[0], config.image_size[1]), device=str(device)))
except Exception as e:
    print(f"Could not display torchinfo summary: {e}")
    print("Model created successfully without detailed summary")

# Get model information from extracted module
model_info = pytorch_model.get_model_info()
print(f"\nModel Information:")
print(f"  Total parameters: {model_info['total_parameters']:,}")
print(f"  Trainable parameters: {model_info['trainable_parameters']:,}")
print(f"  Model size: {model_info['model_size_mb']:.1f} MB")
print(f"  Architecture: {model_info['architecture']}")

# Test forward pass with sample data
print(f"\nTesting forward pass...")
try:
    sample_batch = next(iter(train_loader))
    with torch.no_grad():
        sample_input = sample_batch[0].to(device)
        sample_output = pytorch_model(sample_input)
        print(f"✅ Forward pass successful: {sample_input.shape} -> {sample_output.shape}")
except Exception as e:
    print(f"❌ Forward pass failed: {e}")

print(f"\n✅ Model ready for training using extracted modules")

## Training Setup and Utilities

In [ ]:
# Training management is now handled by the extracted modules
print("Using extracted DeepLearningV1Trainer from src.trainer module")
print("Trainer features:")
print("- Early stopping with configurable patience")
print("- Learning rate scheduling")
print("- Training history tracking")
print("- Best model state preservation")
print("- Comprehensive logging and metrics")
print("- Overfitting detection and analysis")
print("- Memory-efficient batch processing")
print("- GPU/CPU compatibility")

# Display trainer configuration
print(f"\nTrainer configuration:")
print(f"  Early stopping patience: {config.patience} epochs")
print(f"  Minimum improvement delta: {config.min_delta}")
print(f"  Learning rate: {config.learning_rate}")
print(f"  Weight decay: {config.weight_decay}")
print(f"  LR scheduler step size: {config.lr_step_size}")
print(f"  LR scheduler gamma: {config.lr_gamma}")

print(f"\n✅ Training management ready from extracted modules")

## Model Training

In [ ]:
# Create and use the extracted trainer
print("Setting up training using extracted modules...")

# Create classifier and trainer
classifier = DeepLearningV1Classifier(
    model_name="deep-learning-v1",
    version="1.0.0",
    config=config,
    class_names=class_names
)

# Set the PyTorch model
classifier.model = pytorch_model
classifier.num_classes = num_classes

# Create trainer
trainer = DeepLearningV1Trainer(classifier, config)

print(f"Trainer created for {num_classes} classes")
print(f"Using device: {device}")

# Start training using the extracted trainer
print(f"\n🚀 Starting training with extracted trainer...")
print(f"   - Early stopping patience: {config.patience} epochs")
print(f"   - Learning rate: {config.learning_rate}")
print(f"   - Batch size: {config.batch_size}")
print(f"   - Maximum epochs: {config.num_epochs}")

try:
    # The trainer will handle the complete training pipeline
    results = trainer.train(dataset_path)
    
    print(f"\n✅ Training completed using extracted modules!")
    print(f"   - Test accuracy: {results['metrics']['test_accuracy']:.2f}%")
    print(f"   - Best validation accuracy: {results['metrics']['best_val_accuracy']:.2f}%")
    print(f"   - Training time: {results['metrics']['training_time']:.2f} seconds")
    print(f"   - Epochs trained: {results['metrics']['epochs_trained']}")
    
    # Store results for later use
    trained_model = results['model']
    training_metrics = results['metrics']
    training_history = results['training_history']
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    import traceback
    traceback.print_exc()

## Model Evaluation

In [ ]:
# Model evaluation is handled by the extracted trainer
print("Model evaluation completed during training using extracted modules")

if 'training_metrics' in locals():
    print(f"\nFinal Evaluation Results:")
    print(f"  Test Accuracy: {training_metrics['test_accuracy']:.2f}%")
    print(f"  Best Validation Accuracy: {training_metrics['best_val_accuracy']:.2f}%")
    print(f"  Training Samples: {training_metrics['train_samples']:,}")
    print(f"  Validation Samples: {training_metrics['val_samples']:,}")
    print(f"  Test Samples: {training_metrics['test_samples']:,}")
    print(f"  Number of Classes: {training_metrics['num_classes']}")
    print(f"  Training Time: {training_metrics['training_time']:.2f} seconds")
    
    # Plot training history if available
    if 'training_history' in locals():
        print(f"\nPlotting training progress...")
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        epochs = range(1, len(training_history['train_losses']) + 1)
        
        # Loss plot
        ax1.plot(epochs, training_history['train_losses'], 'b-', label='Training Loss', linewidth=2)
        ax1.plot(epochs, training_history['val_losses'], 'r-', label='Validation Loss', linewidth=2)
        ax1.set_title('Training and Validation Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Accuracy plot
        ax2.plot(epochs, training_history['train_accuracies'], 'b-', label='Training Accuracy', linewidth=2)
        ax2.plot(epochs, training_history['val_accuracies'], 'r-', label='Validation Accuracy', linewidth=2)
        ax2.set_title('Training and Validation Accuracy')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy (%)')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print(f"✅ Training progress visualization complete")
else:
    print("❌ Training results not available - run training first")

In [ ]:
# Visualize some test predictions
def visualize_predictions(model, test_loader, class_names, device, num_samples=8):
    """Visualize model predictions on test samples."""
    model.eval()
    
    # Get a batch of test data
    data_iter = iter(test_loader)
    images, labels = next(data_iter)
    
    # Make predictions
    with torch.no_grad():
        images_gpu = images.to(device)
        outputs = model(images_gpu)
        probabilities = F.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)
    
    # Denormalize images for visualization
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    images_denorm = images * std + mean
    images_denorm = torch.clamp(images_denorm, 0, 1)
    
    # Plot predictions
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()
    
    for i in range(min(num_samples, len(images))):
        img = images_denorm[i].permute(1, 2, 0)
        true_label = class_names[labels[i]]
        pred_label = class_names[predicted[i]]
        confidence = probabilities[i][predicted[i]].item()
        
        axes[i].imshow(img)
        
        # Color based on correctness
        color = 'green' if labels[i] == predicted[i] else 'red'
        
        axes[i].set_title(
            f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2f}',
            color=color
        )
        axes[i].axis('off')
    
    plt.suptitle('Test Predictions (Green=Correct, Red=Incorrect)', fontsize=16)
    plt.tight_layout()
    plt.show()

visualize_predictions(model, test_loader, full_dataset.class_names, device)

## Model Analysis and Insights

In [ ]:
# Analyze model performance by class
def analyze_per_class_performance(targets, predictions, class_names):
    """Analyze performance for each class."""
    from sklearn.metrics import precision_recall_fscore_support
    
    precision, recall, f1, support = precision_recall_fscore_support(
        targets, predictions, average=None, labels=range(len(class_names))
    )
    
    # Create DataFrame for easy visualization
    import pandas as pd
    
    performance_df = pd.DataFrame({
        'Class': class_names,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Support': support
    })
    
    print("Per-Class Performance:")
    print(performance_df.round(3))
    
    # Plot metrics
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    metrics = ['Precision', 'Recall', 'F1-Score']
    for i, metric in enumerate(metrics):
        axes[i].bar(class_names, performance_df[metric])
        axes[i].set_title(f'{metric} by Class')
        axes[i].set_ylabel(metric)
        axes[i].set_ylim(0, 1.1)
        
        # Add value labels on bars
        for j, v in enumerate(performance_df[metric]):
            axes[i].text(j, v + 0.02, f'{v:.3f}', ha='center')
    
    plt.tight_layout()
    plt.show()
    
    return performance_df

performance_df = analyze_per_class_performance(targets, predictions, full_dataset.class_names)

In [ ]:
# Feature visualization - show what the model learned
def visualize_conv_filters(model, layer_name='conv1'):
    """Visualize convolutional filters."""
    # Get the layer
    layer = getattr(model, layer_name)
    filters = layer.weight.data.cpu()
    
    # Normalize filters for visualization
    filters = (filters - filters.min()) / (filters.max() - filters.min())
    
    # Plot first 16 filters
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    axes = axes.ravel()
    
    for i in range(min(16, filters.shape[0])):
        # Convert filter to displayable format
        filter_img = filters[i].permute(1, 2, 0)
        
        if filter_img.shape[2] == 3:  # RGB filter
            axes[i].imshow(filter_img)
        else:  # Single channel
            axes[i].imshow(filter_img[:, :, 0], cmap='gray')
        
        axes[i].set_title(f'Filter {i+1}')
        axes[i].axis('off')
    
    plt.suptitle(f'Learned Filters in {layer_name}', fontsize=16)
    plt.tight_layout()
    plt.show()

# Visualize first layer filters
visualize_conv_filters(model, 'conv1')

## Model Integration with Core Framework

In [ ]:
# Model integration is handled by the extracted modules
print("Using extracted DeepLearningV1Classifier from src.classifier module")
print("Classifier features:")
print("- BaseImageClassifier interface compliance")
print("- Dynamic class discovery and configuration")
print("- Preprocessing pipeline with configurable transforms")
print("- Batch prediction support")
print("- Feature map extraction capabilities")
print("- Model serialization and deserialization")
print("- GPU/CPU compatibility")
print("- Comprehensive metadata reporting")

# Show classifier configuration
if 'classifier' in locals():
    print(f"\nClassifier Configuration:")
    print(f"  Model name: {classifier.model_name}")
    print(f"  Version: {classifier.version}")
    print(f"  Number of classes: {classifier.num_classes}")
    print(f"  Input size: {classifier.config.image_size}")
    print(f"  Device: {classifier.device}")
    print(f"  Model loaded: {classifier.is_loaded}")
    
    # Get metadata from extracted classifier
    try:
        metadata = classifier.get_metadata()
        print(f"\nClassifier Metadata:")
        print(f"  Architecture: {metadata.get('architecture', 'N/A')}")
        print(f"  Parameters: {metadata.get('parameters', 'N/A'):,}")
        print(f"  Model size: {metadata.get('model_size_mb', 'N/A'):.1f} MB")
        print(f"  Model type: {metadata.get('model_type', 'N/A')}")
    except Exception as e:
        print(f"Could not retrieve metadata: {e}")

print(f"\n✅ Classifier integration complete using extracted modules")

In [ ]:
# Save and test the model using extracted modules
if 'trained_model' in locals() and 'training_metrics' in locals():
    print("Saving trained model using extracted classifier...")
    
    # Save the model using the extracted classifier
    model_path = "../models/deep_v1_classifier.pth"
    os.makedirs("../models", exist_ok=True)
    
    classifier.save_model(
        model_path,
        model=trained_model,
        class_names=class_names,
        accuracy=training_metrics['test_accuracy'],
        training_history=training_history
    )
    
    print(f"Model saved to {model_path}")
    
    # Test the saved model by loading it fresh
    print("\nTesting model loading and prediction...")
    test_classifier = DeepLearningV1Classifier()
    test_classifier.load_model(model_path)
    
    # Test prediction on a sample image
    try:
        sample_batch = next(iter(test_loader))
        sample_image_tensor = sample_batch[0][0]  # Get first image from batch
        sample_label = sample_batch[1][0].item()  # Get corresponding label
        
        # Convert tensor back to numpy for prediction testing
        # Denormalize the image
        mean = torch.tensor(config.normalize_mean).view(3, 1, 1)
        std = torch.tensor(config.normalize_std).view(3, 1, 1)
        sample_image_denorm = sample_image_tensor * std + mean
        sample_image_denorm = torch.clamp(sample_image_denorm, 0, 1)
        sample_image_np = (sample_image_denorm.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        
        # Make prediction using extracted classifier
        predictions = test_classifier.predict(sample_image_np)
        
        print(f"\nSample prediction test:")
        # Show top 5 predictions
        sorted_preds = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
        for i, (class_name, prob) in enumerate(sorted_preds[:5]):
            print(f"  {i+1}. {class_name}: {prob:.4f}")
        
        print(f"\nActual class: {class_names[sample_label]}")
        predicted_class = max(predictions.items(), key=lambda x: x[1])[0]
        correct = predicted_class == class_names[sample_label]
        print(f"Predicted class: {predicted_class}")
        print(f"Prediction correct: {'✅' if correct else '❌'}")
        
    except Exception as e:
        print(f"Error in prediction test: {e}")
    
    # Register model in registry
    print(f"\nRegistering model in registry...")
    registry = ModelRegistry()
    metadata = ModelMetadata(
        name="deep-learning-v1",
        version="1.0.0",
        model_type="deep_v1",
        accuracy=training_metrics['test_accuracy'] / 100.0,  # Convert to decimal
        training_date="2024-01-01",
        model_path=model_path,
        config={
            "architecture": "Custom CNN",
            "num_classes": len(class_names),
            "input_size": f"{config.image_size[0]}x{config.image_size[1]}x{config.input_channels}",
            "epochs_trained": training_metrics['epochs_trained'],
            "optimizer": "Adam",
            "learning_rate": config.learning_rate,
            "batch_size": config.batch_size
        },
        performance_metrics={
            "test_accuracy": training_metrics['test_accuracy'] / 100.0,
            "best_val_accuracy": training_metrics['best_val_accuracy'] / 100.0,
            "training_time": training_metrics['training_time'],
            "training_samples": training_metrics['train_samples'],
            "test_samples": training_metrics['test_samples']
        }
    )
    
    registry.register_model(metadata)
    print(f"✅ Model registered with test accuracy: {training_metrics['test_accuracy']:.2f}%")
    print(f"✅ Training completed successfully using extracted modules!")
    print(f"   Total classes: {len(class_names)}")
    print(f"   Model path: {model_path}")
    
else:
    print("❌ No trained model available - run training first")

## Model Comparison and Analysis

In [ ]:
# Compare with shallow learning if available
def compare_with_shallow_learning():
    """Compare performance with shallow learning baseline."""
    try:
        # Try to load shallow learning results for comparison
        shallow_registry = registry.get_model("shallow-classifier")
        
        if shallow_registry:
            shallow_accuracy = shallow_registry.accuracy * 100
            deep_accuracy = test_accuracy
            
            print(f"\nModel Comparison:")
            print(f"Shallow Learning Accuracy: {shallow_accuracy:.2f}%")
            print(f"Deep Learning v1 Accuracy: {deep_accuracy:.2f}%")
            print(f"Improvement: {deep_accuracy - shallow_accuracy:.2f}%")
            
            # Plot comparison
            models = ['Shallow Learning', 'Deep Learning v1']
            accuracies = [shallow_accuracy, deep_accuracy]
            
            plt.figure(figsize=(8, 6))
            bars = plt.bar(models, accuracies, color=['skyblue', 'lightcoral'])
            plt.title('Model Performance Comparison')
            plt.ylabel('Accuracy (%)')
            plt.ylim(0, 100)
            
            # Add value labels on bars
            for bar, acc in zip(bars, accuracies):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                        f'{acc:.1f}%', ha='center', va='bottom')
            
            plt.tight_layout()
            plt.show()
        else:
            print("Shallow learning model not found for comparison.")
            
    except Exception as e:
        print(f"Could not compare with shallow learning: {e}")

compare_with_shallow_learning()

## Summary and Insights

### Model Architecture:
- **Custom CNN**: 5 convolutional layers with batch normalization (optimized for 128x128 input)
- **Feature Progression**: 3 → 32 → 64 → 128 → 256 → 512 channels
- **Input Resolution**: 128x128x3 (sweet spot between detail and model capacity)
- **Regularization**: Dropout, batch normalization, data augmentation
- **Global Average Pooling**: Reduces overfitting compared to fully connected layers

### Training Strategy:
- **Optimal Resolution**: 128x128 balances detail capture with computational efficiency
- **Memory-Efficient Loading**: On-demand image loading to handle large datasets
- **Data Augmentation**: Random flips, rotations, color jittering
- **Optimization**: Adam optimizer with learning rate scheduling
- **Early Stopping**: Based on validation accuracy
- **Monitoring**: Real-time loss and accuracy tracking

### Resolution Analysis Results:
1. **64x64**: Best performance initially due to simpler learning task
2. **256x256**: Too much detail for current model capacity, may overfit
3. **128x128**: Expected sweet spot - enough detail without overwhelming the model

### Key Benefits of 128x128:
- **Balanced Complexity**: 4x more detail than 64x64, but manageable for training
- **Better Memory Usage**: More efficient than 256x256, allows larger batch sizes
- **Optimal Learning**: Sufficient detail for discrimination without overfitting
- **Faster Training**: Quicker than 256x256 while maintaining good accuracy

### Memory Considerations:
- **Image Size**: 128x128 = 4x more pixels than 64x64
- **Batch Size**: Can maintain batch_size=8 with good GPU memory usage
- **Model Complexity**: 5 layers handle 128x128 efficiently
- **Training Speed**: Good balance between speed and accuracy

### Expected Performance:
- **Better than 64x64**: More detail for fine-grained classification
- **Better than 256x256**: Avoids overfitting and excessive computational load
- **Optimal Training**: Model capacity matches input complexity
- **Good Convergence**: Expected stable training with good final accuracy

### Production Readiness:
- Model integrated with core framework
- Saved in portable format for deployment
- Compatible with ensemble classifier
- Ready for API integration with 128x128 input standardization

### Next Steps:
1. **Performance Validation**: Confirm 128x128 outperforms both 64x64 and 256x256
2. **Hyperparameter Tuning**: Optimize learning rate and batch size for 128x128
3. **Architecture Refinement**: Consider adding skip connections if needed
4. **Ensemble Integration**: Combine with other models for maximum accuracy